In [92]:
import numpy as np
import pandas as pd
sales_data = pd.read_csv('Train_UWu5bXk.csv')

In [93]:
meanbytype=sales_data.groupby('Item_Type').Item_Weight.transform('mean')
sales_data['Item_Weight'].fillna(meanbytype,inplace=True)
sales_data['Outlet_Size'].fillna('Missing',inplace=True)
# get the age of Outlet
import datetime 
datetime.datetime.now().year
sales_data['Outlet_Age']=sales_data.Outlet_Establishment_Year.sub(datetime.datetime.now().year).abs()
#Encode outlet size ordinally
scale_size={'Missing':-1,'Small':1,'Medium':2,'High':3}
sales_data['Outlet_Size']=sales_data['Outlet_Size'].replace(scale_size)
#clean fat content column
sales_data.Item_Fat_Content=sales_data.Item_Fat_Content.str.upper()
scale_fat_content={'LOW FAT':0,'LF':0,'REGULAR':1,'REG':1,'HIGH FAT':2,'HF':2}
sales_data['Item_Fat_Content']=sales_data['Item_Fat_Content'].replace(scale_fat_content)
sales_data.Outlet_Location_Type=pd.to_numeric(sales_data.Outlet_Location_Type.str.replace('Tier ',''))
scale_fat_outlet_type={'Grocery Store':0,'Supermarket Type1':1,'Supermarket Type2':2,'Supermarket Type3':3}
sales_data['Outlet_Type']=sales_data['Outlet_Type'].replace(scale_fat_outlet_type)
sales_data=pd.concat([sales_data,pd.get_dummies(sales_data.Item_Type,prefix='Item_Type')],axis=1)
sales_data['Item_Ident_prefix']=sales_data.Item_Identifier.str.slice(start=0,stop=2)
sales_data=pd.concat([sales_data,pd.get_dummies(sales_data.Item_Ident_prefix,prefix='Item_Identifier')],axis=1)
sales_data=pd.concat([sales_data,pd.get_dummies(sales_data.Outlet_Identifier,prefix='Outlet_Identifier')],axis=1)
sales_data=sales_data.drop(['Item_Identifier','Outlet_Establishment_Year','Item_Type','Outlet_Identifier','Item_Ident_prefix'],axis=1)

In [94]:
y=sales_data['Item_Outlet_Sales'].values
X=sales_data.drop(['Item_Outlet_Sales'],axis=1).values

# from sklearn.preprocessing import StandardScaler
# ss=StandardScaler()
# X_std=ss.fit_transform(X)
# from sklearn.preprocessing import PolynomialFeatures
# pf=PolynomialFeatures(3)
# X_Poly_test=pf.fit_transform(X_std)

from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.25,random_state=0)

In [95]:
from sklearn.ensemble import GradientBoostingRegressor
rfc=GradientBoostingRegressor(loss='ls',
  max_depth= 3,
  min_samples_leaf= 100,
  min_samples_split= 799,
  n_estimators=58)
rfc.fit(xtrain,ytrain)
# print(rfc.score(xtrain,ytrain))
# print(rfc.score(xtest,ytest))
pred_train=rfc.predict(xtrain)
pred_test=rfc.predict(xtest)
from sklearn.metrics import mean_squared_error
lin_rmse_train = np.sqrt(mean_squared_error(ytrain,pred_train))
lin_rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))
print("Train Error",lin_rmse_train,"Test Error",lin_rmse_test)

Train Error 1055.1424954648342 Test Error 1095.8731892139044


In [9]:
sales_data.columns

Index(['Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_MRP',
       'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type',
       'Item_Outlet_Sales', 'Outlet_Age', 'Item_Type_Baking Goods',
       'Item_Type_Breads', 'Item_Type_Breakfast', 'Item_Type_Canned',
       'Item_Type_Dairy', 'Item_Type_Frozen Foods',
       'Item_Type_Fruits and Vegetables', 'Item_Type_Hard Drinks',
       'Item_Type_Health and Hygiene', 'Item_Type_Household', 'Item_Type_Meat',
       'Item_Type_Others', 'Item_Type_Seafood', 'Item_Type_Snack Foods',
       'Item_Type_Soft Drinks', 'Item_Type_Starchy Foods',
       'Item_Identifier_DR', 'Item_Identifier_FD', 'Item_Identifier_NC',
       'Outlet_Identifier_OUT010', 'Outlet_Identifier_OUT013',
       'Outlet_Identifier_OUT017', 'Outlet_Identifier_OUT018',
       'Outlet_Identifier_OUT019', 'Outlet_Identifier_OUT027',
       'Outlet_Identifier_OUT035', 'Outlet_Identifier_OUT045',
       'Outlet_Identifier_OUT046', 'Outlet_Identifier_OUT049'],
      dt

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})
sales_data.hist(bins=50,figsize=(30,30),xlabelsize=20, ylabelsize=25 , grid=True)
plt.show()

In [ ]:
plt.rcParams.update({'font.size': 50})
from pandas.plotting import scatter_matrix

# attributes = ['Item_MRP' , 'Item_Visibility' , 'Item_Outlet_Sales']# , 'Outlet_Establishment_Year']
# help(scatter_matrix)
scat=scatter_matrix(sales_data[['Item_Visibility','Item_Outlet_Sales']], figsize=(50,40))
# scat=scatter_matrix(sales_data[['Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type','Item_Outlet_Sales']], figsize=(50,40))

In [82]:
#n_estimators=[0.01,0.25,0.5,0.75,1]
n_estimators=['ls', 'lad', 'huber', 'quantile']
df=pd.DataFrame(columns=['Parameter','train_error','test_error','diff'])
for val in n_estimators:#range(2,300,10):
    rfc=GradientBoostingRegressor(loss=val,n_estimators=60,max_depth=3,min_samples_split=775,min_samples_leaf=102)
    rfc.fit(xtrain,ytrain)
    pred_train=rfc.predict(xtrain)
    pred_test=rfc.predict(xtest)
    lin_rmse_train = np.sqrt(mean_squared_error(ytrain,pred_train))
    lin_rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))
#     print('Parameter',val,'train_error',lin_rmse_train,'test_error',lin_rmse_test)
    df=df.append(pd.DataFrame({'Parameter':[val],'train_error':[lin_rmse_train],'test_error':[lin_rmse_test],'diff':[lin_rmse_test-lin_rmse_train]}),ignore_index=True)
df

,Parameter,train_error,test_error,diff
0,ls,1054.954495,1095.845962,40.891468
1,lad,1072.175265,1104.952226,32.776960
2,huber,1057.864629,1100.502951,42.638322
3,quantile,1749.628140,1780.929738,31.301598


In [86]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import regression
from sklearn.metrics import confusion_matrix


def dt_param_grid(X, y, nfolds):
    
    param_grid = {'loss':['ls', 'lad', 'huber', 'quantile'],
                  'max_depth': [2,3,4,5,6], 
                  'min_samples_split': [760,770,775,780,799],
                  'min_samples_leaf':[92,95,100,102,105],
                  'n_estimators':[58,59,60,61,62]
                 }
    ##grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=nfolds)
    
    grid_search = GridSearchCV(GradientBoostingRegressor(), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    print(grid_search.best_params_)
    
    print("Detailed classification report:")
    print()
    y_true, y_pred = y, grid_search.predict(X)
    print(regression.mean_squared_error(y_true, y_pred))
    print()
    
    return grid_search.best_params_,grid_search.score(X,y)

In [90]:
dt_param_grid(X,y,10)

{'loss': 'ls', 'max_depth': 3, 'min_samples_leaf': 100, 'min_samples_split': 799, 'n_estimators': 58}
Detailed classification report:

1126914.2280779937



({'loss': 'ls',
  'max_depth': 3,
  'min_samples_leaf': 100,
  'min_samples_split': 799,
  'n_estimators': 58},
 0.6129835444896026)

In [133]:
sales_data_test=pd.read_csv('Test_u94Q5KV.csv')
sales_data_tmp=sales_data_test
meanbytype=sales_data_test.groupby('Item_Type').Item_Weight.transform('mean')
sales_data_test['Item_Weight'].fillna(meanbytype,inplace=True)
sales_data_test['Outlet_Size'].fillna('Missing',inplace=True)
# get the age of Outlet
import datetime 
datetime.datetime.now().year
sales_data_test['Outlet_Age']=sales_data_test.Outlet_Establishment_Year.sub(datetime.datetime.now().year).abs()
#Encode outlet size ordinally
scale_size={'Missing':1,'Small':1,'Medium':2,'High':3}
sales_data_test['Outlet_Size']=sales_data_test['Outlet_Size'].replace(scale_size)
#clean fat content column
sales_data_test.Item_Fat_Content=sales_data_test.Item_Fat_Content.str.upper()
scale_fat_content={'LOW FAT':0,'LF':0,'REGULAR':1,'REG':1}
sales_data_test['Item_Fat_Content']=sales_data_test['Item_Fat_Content'].replace(scale_fat_content)
sales_data_test.Outlet_Location_Type=pd.to_numeric(sales_data_test.Outlet_Location_Type.str.replace('Tier ',''))
scale_fat_outlet_type={'Grocery Store':0,'Supermarket Type1':1,'Supermarket Type2':2,'Supermarket Type3':3}
sales_data_test['Outlet_Type']=sales_data_test['Outlet_Type'].replace(scale_fat_outlet_type)
# sales_data_test.Item_Type=LabEncoder.fit_transform(sales_data_test.Item_Type)
sales_data_test=pd.concat([sales_data_test,pd.get_dummies(sales_data_test.Item_Type,prefix='Item_Type')],axis=1)
# sales_data_test=pd.concat([sales_data_test,pd.get_dummies(sales_data_test.Item_Type,prefix='Item_Identifier')],axis=1)
sales_data_test['Item_Ident_prefix']=sales_data_test.Item_Identifier.str.slice(start=0,stop=2)
sales_data_test=pd.concat([sales_data_test,pd.get_dummies(sales_data_test.Item_Ident_prefix,prefix='Item_Identifier')],axis=1)
# sales_data_test['Item_Ident_post']=sales_data_test.Item_Identifier.str.slice(start=2,stop=3)
sales_data_test=pd.concat([sales_data_test,pd.get_dummies(sales_data_test.Outlet_Identifier,prefix='Outlet_Identifier')],axis=1)
sales_data_test=sales_data_test.drop(['Item_Identifier','Outlet_Establishment_Year','Item_Type','Outlet_Identifier','Item_Ident_prefix'],axis=1)
# from sklearn.preprocessing import StandardScaler
# ss=StandardScaler()
# X_std=ss.fit_transform(sales_data_test)
# from sklearn.preprocessing import PolynomialFeatures
# pf=PolynomialFeatures(3)
# X_Poly_test=pf.fit_transform(X_std)

In [108]:
from sklearn.ensemble import GradientBoostingRegressor
rfc=GradientBoostingRegressor(loss='ls',
  max_depth= 3,
  min_samples_leaf= 100,
  min_samples_split= 799,
  n_estimators=58)
rfc.fit(xtrain,ytrain)
# print(rfc.score(xtrain,ytrain))
# print(rfc.score(xtest,ytest))
pred_train=rfc.predict(xtrain)
pred_test=rfc.predict(xtest)
from sklearn.metrics import mean_squared_error
lin_rmse_train = np.sqrt(mean_squared_error(ytrain,pred_train))
lin_rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))
print("Train Error",lin_rmse_train,"Test Error",lin_rmse_test)

Train Error 1055.1424954648342 Test Error 1095.8731892139044


In [98]:
from sklearn.ensemble import RandomForestRegressor
rfr=RandomForestRegressor()
rfr.fit(xtrain,ytrain)
pred_train=rfr.predict(xtrain)
pred_test=rfr.predict(xtest)
lin_rmse_train = np.sqrt(mean_squared_error(ytrain,pred_train))
lin_rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))
print("Train Error",lin_rmse_train,"Test Error",lin_rmse_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Train Error 500.8595459622268 Test Error 1175.135881473382


In [119]:
help(RandomForestRegressor)

Help on class RandomForestRegressor in module sklearn.ensemble.forest:

class RandomForestRegressor(ForestRegressor)
 |  RandomForestRegressor(n_estimators='warn', criterion='mse', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False)
 |  
 |  A random forest regressor.
 |  
 |  A random forest is a meta estimator that fits a number of classifying
 |  decision trees on various sub-samples of the dataset and uses averaging
 |  to improve the predictive accuracy and control over-fitting.
 |  The sub-sample size is always the same as the original
 |  input sample size but the samples are drawn with replacement if
 |  `bootstrap=True` (default).
 |  
 |  Read more in the :ref:`User Guide <forest>`.
 |  
 |  Parameters
 |  ----------
 |  n_estimators : integer, optional (de

In [ ]:
for i in range(100,200,5):
    rfr=RandomForestRegressor(warm_start=True,n_estimators =100,max_depth=6,min_samples_split=100,min_samples_leaf=6)
    rfr.fit(xtrain,ytrain)
    pred_train=rfr.predict(xtrain)
    pred_test=rfr.predict(xtest)
    lin_rmse_train = np.sqrt(mean_squared_error(ytrain,pred_train))
    lin_rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))
    print("Param",i,"Train Error",lin_rmse_train,"Test Error",lin_rmse_test)

In [114]:
rfc=GradientBoostingRegressor(loss='ls',
  max_depth= 3,
  min_samples_leaf= 100,
  min_samples_split= 799,
  n_estimators=58)
rfc.fit(X,y)
pred=rfc.predict(X)
lin_rmse_train = np.sqrt(mean_squared_error(y,pred))
lin_rmse_train

1061.5621640196082

In [137]:
rfr=RandomForestRegressor(n_estimators =100,max_depth=5,min_samples_split=100,min_samples_leaf=6)
rfr.fit(X,y)
pred=rfc.predict(X)
lin_rmse_train = np.sqrt(mean_squared_error(y,pred))
print("Train Error",lin_rmse_train)

Train Error 1065.4723581077258


In [134]:
sales_data_tmp['Item_Outlet_Sales']=rfr.predict(sales_data_test)

In [135]:
final_Output=sales_data_tmp[['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales']]
final_Output.to_csv('BigDataMart_Predicted3.csv',index=False)